We will be using the unsructured-api for retrieving the tables from the pdf.

In [10]:
# Import necessary libraries
import os
import pandas as pd

from unstructured_ingest.v2.pipeline.pipeline import Pipeline
from unstructured_ingest.v2.interfaces import ProcessorConfig
from unstructured_ingest.v2.processes.connectors.local import (
    LocalIndexerConfig,
    LocalDownloaderConfig,
    LocalConnectionConfig,
    LocalUploaderConfig
)
from unstructured_ingest.v2.processes.partitioner import PartitionerConfig

# Main execution block
if __name__ == "__main__":

    # 1. Pipeline Configuration
    # This block defines the configuration for the data processing pipeline.
    Pipeline.from_configs(
        context = ProcessorConfig(),
        indexer_config = LocalIndexerConfig(input_path = "/Data/example_pdf/2023q4-alphabet-earnings-release.pdf"),
        downloader_config = LocalDownloaderConfig(),
        source_connection_config = LocalConnectionConfig(),
        partitioner_config  =PartitionerConfig(
            partition_by_api = True,
            api_key = "SCZU2XpxMZq8OnXgorpBg0bLC16Tnq",
            partition_endpoint = "https://api.unstructured.io/general/v0/general",
            strategy = "hi_res",
            additional_partition_args = {
                "split_pdf_page": True,
                "split_pdf_allow_failed": True,
                "split_pdf_concurrency_level": 15
            }
        ),
        uploader_config=LocalUploaderConfig(output_dir = "../Data/processed_pdf")
    ).run()


Overriding of current TracerProvider is not allowed
2025-01-13 13:57:12,173 MainProcess INFO     created index with configs: {"input_path": "\\Data\\example_pdf\\2023q4-alphabet-earnings-release.pdf", "recursive": false}, connection configs: {"access_config": "**********"}
2025-01-13 13:57:12,179 MainProcess INFO     Created download with configs: {"download_dir": null}, connection configs: {"access_config": "**********"}
2025-01-13 13:57:12,181 MainProcess INFO     created partition with configs: {"strategy": "hi_res", "ocr_languages": null, "encoding": null, "additional_partition_args": {"split_pdf_page": true, "split_pdf_allow_failed": true, "split_pdf_concurrency_level": 15}, "skip_infer_table_types": null, "fields_include": ["element_id", "text", "type", "metadata", "embeddings"], "flatten_metadata": false, "metadata_exclude": [], "element_exclude": [], "metadata_include": [], "partition_endpoint": "https://api.unstructured.io/general/v0/general", "partition_by_api": true, "api_ke

In [11]:
# Load Processed Data (after the pipeline execution is successful)
processed_data = pd.read_json("../Data/processed_pdf/2023q4-alphabet-earnings-release.pdf.json")

In [12]:
len(processed_data)

98

In [13]:
tables = processed_data[processed_data["type"] == "Table"]

In [14]:
# 5. Extract Table Text
all_tables = []
for table in tables["metadata"].values:
    all_tables.append(table["text_as_html"])

In [16]:
# 6. Clean HTML Tables (using BeautifulSoup)
from bs4 import BeautifulSoup

soup = BeautifulSoup(all_tables[0], "html.parser")

#  - Remove dollar signs and leading/trailing whitespaces from table data (<td> elements).
for td in soup.find_all("td"):
    if td.string:
        td.string = td.string.replace("$", "").strip()

#  - Remove empty table cells (<td>)
    if not td.string:
        td.decompose()

cleaned_html = soup.prettify() # Format the cleaned HTML for better readability
print(cleaned_html)

<table>
 <thead>
  <tr>
   <th>
   </th>
   <th colspan="2">
    Quarter Ended December 31,
   </th>
   <th colspan="2">
    Year Ended December 31,
   </th>
  </tr>
  <tr>
   <th>
   </th>
   <th>
    2022
   </th>
   <th>
    2023
   </th>
   <th>
    2022
   </th>
   <th>
    2023
   </th>
  </tr>
  <tr>
   <th>
   </th>
   <th>
   </th>
   <th>
    (unaudited)
   </th>
   <th>
   </th>
   <th>
    (unaudited)
   </th>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td>
    Revenues
   </td>
   <td>
    76,048
   </td>
   <td>
    86,310
   </td>
   <td>
    282,836
   </td>
   <td>
    307,394
   </td>
  </tr>
  <tr>
   <td>
    Change in revenues year over year
   </td>
   <td>
    1%
   </td>
   <td>
    13 %
   </td>
   <td>
    10 %
 %
   </td>
   <td>
    9 %
   </td>
  </tr>
  <tr>
   <td>
    Change in constant currency revenues year over year'"
   </td>
   <td>
    7 %
   </td>
   <td>
    13 %
   </td>
   <td>
    14 %
 %
   </td>
   <td>
    10 %
 %
   </td>
  </tr>
  <tr>
   <td>


In [17]:
# Displaying the cleaned HTML in the notebook
from IPython.core.display import HTML
HTML(cleaned_html)

In [18]:
from io import StringIO

In [21]:
# Read the cleaned HTML into a Pandas DataFrame
df_list = pd.read_html(StringIO(cleaned_html))

In [22]:
df = df_list[0]

In [23]:
df

Unnamed: 0_level_0  \
                                  Unnamed: 0_level_1   
                                  Unnamed: 0_level_2   
0                                           Revenues   
1                  Change in revenues year over year   
2  Change in constant currency revenues year over...   
3                                   Operating income   
4                                   Operating margin   
5                        Other income (expense), net   
6                                         Net income   
7                                        Diluted EPS   

  Quarter Ended December 31,             Year Ended December 31,              
                        2022        2023                    2022        2023  
          Unnamed: 1_level_2 (unaudited)      Unnamed: 3_level_2 (unaudited)  
0                      76048       86310                  282836      307394  
1                         1%        13 %                 10 %  %         9 %  
2                        7 %        13 %                 14 %  %     10 %  %  
3                      18160       23697                   74842       84293  
4                    24 %  %     27 %  %                 26 %  %     27 %  %  
5                    (1,013)         715                 (3,514)        1424  
6                      13624       20687                   59972       73795  
7                       1.05      ¢ 1.64                    4.56        5.80